In [1]:
import csv
from datetime import timedelta
import gurobipy as gp 
from gurobipy import GRB
import networkx as nx
import random
import matplotlib.pyplot as plt
import inspect

In [2]:
def hhmm2mins(hhmm):
    h, m = map(int, hhmm.split(':'))
    return h*60 + m

def mins2hhmm(mins):
    h = mins // 60
    m = mins % 60
    return f"{h:02}:{m:02}"

In [3]:
class Service:
    def __init__(self, attrs):
        self.serv_num = int(attrs[0])
        self.train_num = attrs[1]
        self.start_stn = attrs[2]
        self.start_time = hhmm2mins(attrs[3])
        self.end_stn = attrs[4]
        self.end_time = hhmm2mins(attrs[5])
        self.direction = attrs[6]
        self.serv_dur = int(attrs[7])
        self.jurisdiction = attrs[8]
        self.stepback_train_num = attrs[9]
        self.serv_added = False
        self.break_dur = 0
        self.trip_dur = 0

In [4]:
def fetch_data(filename):
    services = []
    with open(filename, 'r') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            services.append(Service(row))
    return services

In [5]:
def draw_graph_with_edges(graph, n=50):
    # Create a directed subgraph containing only the first n edges
    subgraph = nx.DiGraph()
    
    # Add the first n edges and associated nodes to the subgraph
    edge_count = 0
    for u, v in graph.edges():
        if edge_count >= n:
            break
        subgraph.add_edge(u, v)
        edge_count += 1

    # Plotting the directed subgraph
    plt.figure(figsize=(8, 8))
    pos = nx.spring_layout(subgraph)  # Position nodes using the spring layout
    nx.draw_networkx_nodes(subgraph, pos, node_size=50, node_color='red')
    nx.draw_networkx_labels(subgraph, pos, font_size=15)
    nx.draw_networkx_edges(
        subgraph, pos, arrowstyle='->', arrowsize=20, edge_color='blue'
    )
    
    plt.title(f"First {n} Directed Edges of the Network")
    # plt.show()
    plt.savefig(f'first{n}edges.png')


In [6]:
## checking if two services can be connected

def node_legal(service1, service2):
    if service1.stepback_train_num == "No Stepback":
        if service2.train_num == service1.train_num:
            if service1.end_stn == service2.start_stn and 0 <= (service2.start_time - service1.end_time) <= 15:
                return True
        else:
            if (service1.end_stn[:4] == service2.start_stn[:4]) and (service2.start_time >= service1.end_time + 30) and (service2.start_time <= service1.end_time + 150):
                return True
        
    else:
        if service2.train_num == service1.stepback_train_num:
            if (service1.end_stn == service2.start_stn) and (service1.end_time == service2.start_time):
                return True
        else:
            if (service1.end_stn[:4] == service2.start_stn[:4]) and (service2.start_time >= service1.end_time + 30 ) and (service2.start_time <= service1.end_time + 150):
                return True
    return False

In [24]:
# Define the condition for no overlap between two services
def no_overlap(service1, service2):
    return service1.end_time <= service2.start_time

# Step 1: Create the network of duties
def create_duty_graph(services):
    G = nx.DiGraph()

    for i, service1 in enumerate(services):
        G.add_node(service1.serv_num)

    for i, service1 in enumerate(services):
        for j, service2 in enumerate(services):
            if i != j:
                if node_legal(service1, service2):
                    G.add_edge(service1.serv_num, service2.serv_num, weight=service1.serv_dur)
    return G

# Check if a service can be appended to a duty
def can_append(duty, service):
    last_service = duty[-1]
    
    start_end_stn_tf = last_service.end_stn == service.start_stn
    # print(service.start_time, last_service.end_time)
    start_end_time_tf = 5 <= (service.start_time - last_service.end_time) <= 15
    start_end_stn_tf_after_break = last_service.end_stn[:4] == service.start_stn[:4]
    start_end_time_within = 50 <= (service.start_time - last_service.end_time) <= 150

    if last_service.stepback_train_num == "No StepBack":
        start_end_rake_tf = last_service.train_num == service.train_num
    else:
        start_end_rake_tf = last_service.stepback_train_num == service.train_num
    
    # Check for valid conditions and time limits
    if start_end_rake_tf and start_end_stn_tf and start_end_time_tf:
        time_dur = service.end_time - duty[0].start_time
        cont_time_dur = sum([serv.serv_dur for serv in duty])
        if cont_time_dur <= 180 and time_dur <= 445:
            return True
    elif start_end_time_within and start_end_stn_tf_after_break:
        time_dur = service.end_time - duty[0].start_time
        if time_dur <= 445:
            return True
    return False


In [91]:
def generate_initial_feasible_duties_random_from_services(services, num_duties=50):
    feasible_duties = []

    for service1 in services:
        duty = [service1]
        for service2 in services:
            if service1.serv_num != service2.serv_num:
                if can_append(duty, service2):
                    duty.append(service2)
        feasible_duties.append(duty)

    random_duties = random.sample(feasible_duties, num_duties)
    serv_num_duty = []
    
    for duty in random_duties:
        tt = []
        for serv in duty:
            tt.append(serv.serv_num)
        serv_num_duty.append(tt)
    return serv_num_duty

In [21]:
def solve_pricing_problem_with_bellman_ford(graph, dual_values):        # solves shortest path after updating the edge weights
    new_duties = []
    reduced_costs = []
    # graph_copy = graph.copy()

    # Update edge weights based on dual values
    for u, v in graph.edges():
        service_idx = u 
        graph[u][v]['weight'] -= int(dual_values[service_idx])  # Adjust edge weight by dual value

    # Find shortest path using Bellman-Ford from every service (node) in the graph
    for start_service in graph.nodes():
        try:
            length, path = nx.single_source_bellman_ford(graph, start_service, weight='weight')
            
            # Check paths for negative reduced cost (i.e., profitable duties)
            for target_service, cost in length.items():
                if cost < 0:
                    duty_path = path[target_service]
                    new_duties.append(duty_path)  # Add the duty (path) to new duties
                    reduced_costs.append(cost)

        except nx.NetworkXUnbounded:
            print(f"Negative cycle detected starting from service {start_service}")
            continue

    print(f"Duties with negative reduced cost: {len(new_duties)}")
    return new_duties, reduced_costs


In [10]:
# helper functions
def edge_greater_than_zero(graph):
    count = 0
    for u, v, data in graph.edges(data=True):
        if data['weight'] > 0:
            count += 1
    return count

# Picks the duty with the least reduced cost to add to the set, and also returns its reduced cost
def pick_new_duty_and_its_cost(duties, new_duties, red_costs):              # Works on parsing the values returned from bellman ford function
    sorted_pairs = sorted(zip(new_duties, red_costs), key=lambda x: x[1], reverse=True)
    sorted_duties, sorted_costs = zip(*sorted_pairs)                        # Sorted in descending order
    sorted_duties = list(sorted_duties)
    sorted_costs = list(sorted_costs)
    for i in range(len(sorted_duties)-1, -1, -1):
        if sorted_duties[i] not in duties:
            return sorted_duties[i], sorted_costs[i]
    # return sorted_duties, sorted_costs

In [77]:
def solving_RMLP_art_vars_final(services, duties):

    objective = 0
    model = gp.Model("CrewScheduling")
    model.setParam('OutputFlag', 0)

    # Step 1: Define the decision variables
    duty_vars = []
    for i in range(len(duties)):
        duty_vars.append(model.addVar(vtype=GRB.CONTINUOUS, ub=GRB.INFINITY, lb=0, name=f"x{i}"))

    big_penalty = 1e6

    # Step 2: Define artificial variables (same as number of services)
    artificial_vars = []
    for i in range(len(services)):
        artificial_vars.append(model.addVar(vtype=GRB.CONTINUOUS, lb=0, ub=GRB.INFINITY, obj=big_penalty, name=f"artificial_{i}"))

    # Step 3: Set the objective function
    model.setObjective(gp.quicksum(duty_vars) + gp.quicksum(artificial_vars), GRB.MINIMIZE)

    # Step 4: Add service constraints
    service_constraints = []
    for service_idx, service in enumerate(services):
        constr = model.addConstr(
            gp.quicksum(duty_vars[duty_idx] for duty_idx, duty in enumerate(duties) if service.serv_num in duty) + artificial_vars[service_idx] == 1,
            name=f"Service_{service.serv_num}")
        service_constraints.append(constr)

    model.optimize()

    # Step 5: Check the solution and retrieve dual values and selected duties
    if model.status == GRB.INFEASIBLE:
        print('Infeasible problem!')
    elif model.status == GRB.OPTIMAL:
        objective = model.getObjective()
        print("Optimal solution found (with artificial variables)")
        
        # Get the dual variables for each service constraint
        dual_values = [constr.Pi for constr in service_constraints]  # Pi gives the dual variables
        
        selected_duties = [v.varName for v in model.getVars() if v.x > 0.5 and "artificial" not in v.varName]
        
        # Check how many artificial variables are in use
        artificial_in_use = [v.varName for v in artificial_vars if v.x > 0.5]
        print(f"Number of artificial variables in use: {len(artificial_in_use)}")
        print(f"Objective Value: {objective.getValue()}")
        
        # new_duties, reduced_costs = solve_pricing_problem_with_bellman_ford(graph, dual_values)
        # new_duty_to_add = new_duties[reduced_costs.index(min(reduced_costs))]
        
        return selected_duties, dual_values, artificial_in_use
    else:
        print("No optimal solution found.")
        return None, None, None

In [12]:
def column_generation(services, initial_duties, graph):
    duties = initial_duties  # Start with the initial set of duties
    converged = False
    iteration = 0

    while not converged:
        print(f"--- Iteration {iteration} ---")

        # Step 1: Solve the RMLP with the current set of duties
        selected_duties, artificial_in_use, new_duties, reduced_costs = solving_RMLP_art_vars_final(services, duties)
        greater_than_zero = edge_greater_than_zero(graph)
        print("number of artificial variables are: ", len(artificial_in_use))
        
        # Step 2: If there are new duties from the pricing problem, add them to the duty list
        if new_duties != []:
            new_duty, its_cost = pick_new_duty_and_its_cost(duties, new_duties, reduced_costs)
            print(f"Adding new duty: {new_duty}, to the RMLP")
            duties.append(new_duty)  # Add new duties (columns) to the RMLP
            print("new number of duties are: ", len(duties))
            print("new variables added are: ", duties[-1])
            print("reduced cost of the new duty is: ", its_cost)
            print("edges greater than zero are: ",greater_than_zero)
            
            # Continue to the next iteration since we found new duties
            iteration += 1
        else:
            # No new duties with negative reduced cost, meaning the solution has converged
            converged = True
            print("Column generation has converged!")
    
    # Step 3: Final solution with selected duties
    final_solution = selected_duties
    return final_solution


In [132]:
services = fetch_data('StepBackServices.csv')
graph = create_duty_graph(services)
# draw_graph_with_edges(graph,5)

In [133]:
# ITERATION 1

ini_duties = generate_initial_feasible_duties_random_from_services(services, 50)       # works fine

selected_duties, dual_values, art_vars = solving_RMLP_art_vars_final(services, ini_duties)
print(min(dual_values), max(dual_values), dual_values.count(1.0), dual_values.count(0.0), len(selected_duties))
new_duties, reduced_costs = solve_pricing_problem_with_bellman_ford(graph, dual_values)

# draw_graph_with_edges(new_graph)


Optimal solution found (with artificial variables)
Number of artificial variables in use: 775
Objective Value: 810.0
-5.0 1.0 887 15 35
Duties with negative reduced cost: 0


In [ ]:
for _ in range(5):
    selected_duties, dual_values, art_vars = solving_RMLP_art_vars_final(services, ini_duties)
    print(min(dual_values), max(dual_values), dual_values.count(1.0), dual_values.count(0.0), len(selected_duties))
    new_duties, reduced_costs = solve_pricing_problem_with_bellman_ford(graph, dual_values)

In [15]:
new_duty, its_cost = pick_new_duty_and_its_cost(ini_duties, new_duties, reduced_costs)
# print(f"new duty is: {new_duty}, and its cost is: {its_cost}")
print(len(new_duty))



ValueError: not enough values to unpack (expected 2, got 0)

In [151]:
from gurobipy import Model, GRB, quicksum

def solve_master_lp(services, duties, duty_costs, graph=None):
    """
    Solves the master LP for one iteration of column generation.

    Args:
        services (list): List of all services to be covered.
        duties (list): List of current duties (each duty contains 'services' it covers and its 'cost').
        graph (networkx.DiGraph, optional): NetworkX graph if needed for the next step.

    Returns:
        dict: {
            "dual_values": Dual values for each service (to use in the pricing problem),
            "primal_solution": Solution for the current duties,
            "objective_value": Optimal value of the master LP,
            "RMP_model": Gurobi model object for potential debugging,
        }
    """
    # Create Gurobi model
    model = Model("MasterLP")
    
    # Add variables for each duty (decision variables)
    duty_vars = {}
    for idx, cost in enumerate(duty_costs):
        duty_vars[idx] = model.addVar(
            vtype=GRB.CONTINUOUS, 
            lb=0, 
            obj=cost, 
            name=f"duty_{idx}"
        )

    artificial_vars = {}
    for service in services:
        artificial_vars[service] = model.addVar(
            vtype=GRB.CONTINUOUS, 
            lb=0, 
            obj=1e6,  # High penalty
            name=f"artificial_{service}"
        )
    
    # Add constraints: Each service must be covered exactly once
    constraints = {}
    for service in services:
        constraints[service] = model.addConstr(
            quicksum(duty_vars[idx] for idx, duty in enumerate(duties) if service.serv_num in duty) + artificial_vars[service] == 1,
            name=f"cover_{service}"
        )
    
    # Set objective: Minimize total cost
    model.setObjective(
        quicksum(duty_vars[idx] * cost for idx, cost in enumerate(duty_costs)) +
        quicksum(artificial_vars[service] * 1e6 for service in services),
        GRB.MINIMIZE
    )
    
    # Optimize the master LP
    model.optimize()
    
    # Retrieve dual values from constraints
    dual_values = {service.serv_num: constraints[service].Pi for service in services}
    
    # Retrieve the primal solution for the duty variables
    primal_solution = {f"duty_{idx}": var.X for idx, var in duty_vars.items()}
    
    # Return results
    return {
        "dual_values": dual_values,
        "primal_solution": primal_solution,
        "objective_value": model.ObjVal,
        "RMP_model": model
    }

def get_duty_cost(duty):
    cost = 0
    for service in duty:
        edges_from_service = graph.edges(service, data=True)
        for _, _, edge_data in edges_from_service:
            cost += edge_data['weight']
            break
    return cost

duty_costs = []                     # some services are end services so they dont have edges, take that into account later on
for duty in ini_duties:
    duty_costs.append(get_duty_cost(duty))

print(duty_costs)


In [ ]:
def solve_pricing_problem(services, graph, dual_values, source, sink):
    """
    Solves the pricing problem to generate new duties with negative reduced cost.

    Args:
        services (list): List of Service objects.
        graph (networkx.DiGraph): Directed graph with edge weights representing service durations.
        dual_values (dict): Dual values for each service number (node).
        source (int): Starting service number for the duty.
        sink (int): Ending service number for the duty.

    Returns:
        list: List of new duties with negative reduced cost. Each duty is a dictionary:
              {"path": [...], "reduced_cost": float, "cost": float}
    """
    # Create a copy of the graph to adjust weights
    modified_graph = graph.copy()
    
    # Adjust edge weights based on dual values
    for u, v, data in modified_graph.edges(data=True):
        if u in dual_values:
            data['weight'] -= dual_values[u]  # Modify weight using dual value of the source node
    
    # Find the shortest path from source to sink
    try:
        shortest_path = nx.shortest_path(
            modified_graph, source=source, target=sink, weight='weight'
        )
        # Calculate original path cost
        path_cost = sum(
            graph[u][v]['weight'] for u, v in zip(shortest_path[:-1], shortest_path[1:])
        )
        # Calculate reduced cost
        reduced_cost = path_cost - sum(dual_values.get(node, 0) for node in shortest_path)
        
        # Check if reduced cost is negative
        if reduced_cost < 0:
            return [{
                "path": shortest_path,
                "reduced_cost": reduced_cost,
                "cost": path_cost
            }]
        else:
            return []  # No duties with negative reduced cost
    except nx.NetworkXNoPath:
        return []  # No feasible path found


In [161]:
all_values = solve_master_lp(services, ini_duties, duty_costs, graph)

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1255U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 934 rows, 984 columns and 1150 nonzeros
Model fingerprint: 0xaaf31818
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e+01, 1e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 919 rows and 960 columns
Presolve time: 0.01s
Presolved: 15 rows, 24 columns, 47 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.6500622e+08   8.000000e+00   0.000000e+00      0s
Extra simplex iterations after uncrush: 6
      16    7.7300574e+08   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.02 seconds (0.00 work units)
Optimal objective  7.730057440e+08


In [150]:
min_dual = float('inf')
for service, dual in all_values['dual_values'].items():
    min_dual = min(min_dual, dual)
print(min_dual)

-5999891.0
